---
title: "Step-by-Step Development of a Molecular Dynamics Simulation"
author: "Frank Cichos"
jupyter: python3
format:
  html:
    code-fold: true
crossref:
  fig-title: Figure     # (default is "Figure")
  tbl-title: Tbl     # (default is "Table")
  title-delim: "—"   # (default is ":")
  fig-prefix: "Figure"
  eq-prefix: Eq.
  chapters: true
---


So far, we have already developed a basic molecular dynamics simulation for simple atoms and interactions by so-called van der Waals forces. Since we are already nearing the end of the course, we would at least like to add some more complexity to the simulation. In this seminar, we would like to make molecules by binding two atoms together. We will then simulate the vibration of these molecules by solving the equations of motion for the atoms.

## Bond vibrations
A bond vibration is modeled as a harmonic oscillator where two atoms are coupled by a spring force with spring constant $k$. The bond force on atom $i$ due to a bond with atom $j$ is then given by

$$\vec{F}_{bond}^i = -k(\vec{r}_{ij} - \vec{r}_0)$$

where $\vec{r}_{ij}$ is the vector between atom $i$ and atom $j$ and $\vec{r}_0$ is the equilibrium bond length vector. In reality bonds are not harmonic oscillators and deformations lead to a non-linear force curve. However, for small deviations from the equilibrium bond length, the harmonic approximation is quite good. The spring constant $k$ determines the strength of the bond and therefore the frequency $\omega$ of the bond vibration

$$\omega = \sqrt{\frac{k}{\mu}}$$

where $\mu = \frac{m_1 m_2}{m_1 + m_2}$ is the reduced mass of the two bonded atoms with masses $m_1$ and $m_2$.

### Reduced units

As we have defined reduced units for our simulation based on the LJ parameters, we need to express the bond parameters in these units.

#### Bond length

The bond length $r_0$ is expressed in terms of the Lennard-Jones length scale $\sigma$. For the bond between two hydrogen atoms, the equilibrium bond length is approximately $0.74\sigma$.

$$ r_{\text{bond}}^* = \frac{r_{\text{bond}}}{\sigma_{\text{H}}} = \frac{0.74}{1.0} = 0.74 $$

#### Spring constant

The spring constant $k$ is expressed in reduced units based on the LJ energy scale $\epsilon$ and length scale $\sigma$. For a hydrogen molecule, the spring constant is approximately $440$ in reduced units.

$$ k_{\text{spring}}^* = \frac{k \sigma_{\text{H}}^2}{\epsilon_{\text{H}}} $$


## Equations of motion

The equations of motion for the two atoms need to account for both the bond forces and the van der Waals forces. The total force on each atom is the sum of these contributions:

$$m_1 \ddot{\vec{r}}_1 = \vec{F}_{bond}^1 + \vec{F}_{vdW}^1$$

$$m_2 \ddot{\vec{r}}_2 = \vec{F}_{bond}^2 + \vec{F}_{vdW}^2$$

where $\vec{r}_1$ and $\vec{r}_2$ are the positions of the two atoms, $\vec{F}_{bond}$ are the bond forces described above, and $\vec{F}_{vdW}$ are the van der Waals forces we implemented earlier. We can rewrite these equations as a system of first-order differential equations

$$\dot{\vec{v}}_1 = \frac{\vec{F}_{bond}^1 + \vec{F}_{vdW}^1}{m_1}$$

$$\dot{\vec{v}}_2 = \frac{\vec{F}_{bond}^2 + \vec{F}_{vdW}^2}{m_2}$$

$$\dot{\vec{r}}_1 = \vec{v}_1$$

$$\dot{\vec{r}}_2 = \vec{v}_2$$

The van der Waals forces act between atoms of different molecules while the bond forces act between atoms within the same molecule. Both types of forces are calculated at each timestep and combined to determine the total force on each atom.


## Implementation

### The Bond class
As we wrote the simulation quite modular, we can now easily add the bond forces to the simulation. To do so, we will first introduce a new class `Bond` that will store the bond parameters and the atoms involved in the bond.


```python
class Bond:
    def __init__(self, atom1, atom2, k, r0):
        self.atom1 = atom1
        self.atom2 = atom2
        self.k = k      # spring constant
        self.r0 = r0    # equilibrium bond length
```

This bond class will be used to store the bond parameters for each bond in the simulation.


### The DiatomicMolecule class

While this is just a bond between two atoms, which can be also part of a more complicated molecule, we also need to define a molecule class that will store the atoms and bonds that make up the molecule.

```python
class DiatomicMolecule:
    def __init__(self, atom1, atom2, bond):
        self.atom1 = atom1
        self.atom2 = atom2
        self.bond = bond
```

This class will store the two atoms and the bond between them.


### The ForceField class update

We will also need to update the `ForceField` class to store the bond parameters and to calculate the bond forces. We will add a list of bonds to the force field and a method to calculate the bond forces.

```python
class ForceField:
    def __init__(self):
        self.parameters = {
            'C': {'epsilon': 1.615, 'sigma': 1.36},
            'H': {'epsilon': 1.0, 'sigma': 1.0},
            'O': {'epsilon': 1.846, 'sigma': 3.0},
        }
        self.bond_parameters = {
            ('H', 'H'): {'k': 500.0, 'r0': 0.74},  # Example parameters for H2
            ('O', 'H'): {'k': 550.0, 'r0': 0.96},  # Example parameters for OH bond
        }
        self.box_size = None

    def calculate_bond_force(self, bond):
        """Calculate harmonic bond force"""
        r = self.minimum_image_distance(bond.atom1.position, bond.atom2.position)
        r_mag = np.linalg.norm(r)

        # F = -k(r - r0)∙(r/|r|)
        force_mag = -bond.k * (r_mag - bond.r0)
        force = force_mag * r/r_mag
        return force


    # previous code goes here
```

The new addition provides the bond parameters

```python
self.bond_parameters = {
    ('H', 'H'): {'k': 500.0, 'r0': 0.74},  # Example parameters for H2
    ('O', 'H'): {'k': 550.0, 'r0': 0.96},  # Example parameters for OH bond
}
```

which are exemplary values for the bond between two hydrogen atoms and between an oxygen and a hydrogen atom. The method `calculate_bond_force` calculates the bond force between two atoms given a bond object.

```python
def calculate_bond_force(self, bond):
    """Calculate harmonic bond force"""
    r = self.minimum_image_distance(bond.atom1.position, bond.atom2.position)
    r_mag = np.linalg.norm(r)

    # F = -k(r - r0)∙(r/|r|)
    force_mag = -bond.k * (r_mag - bond.r0)
    force = force_mag * r/r_mag
    return force
```

As a first step, this function calculates the distance vector between the two atoms and its magnitude. The force is then calculated as $F = -k(r - r_0) \cdot (r/|r|)$, where $r$ is the distance vector, $r_0$ is the equilibrium bond length, and $k$ is the spring constant. The force is then returned as a vector.


## MD Simulation class update

The MDSimulation class is the controller of the simulation and needs to be updated to include the molecules and bond forces. We will add a list of molecules to the simulation and update the `calculate_forces` method to include the bond forces. The method to update the positions and velocities of the atoms will stay the same as we only need to add the bond forces to the total force calculation.

```python
class MDSimulation:
    def __init__(self, molecules, forcefield, timestep, box_size):
        self.molecules = molecules
        self.atoms = [atom for mol in molecules for atom in [mol.atom1, mol.atom2]]
        self.forcefield = forcefield
        self.forcefield.box_size = box_size
        self.timestep = timestep
        self.box_size = np.array(box_size)
        self.energy_history = []

    def calculate_forces(self):
        # Reset all forces
        for atom in self.atoms:
            atom.reset_force()

        # Calculate bonded forces
        for molecule in self.molecules:
            force = self.forcefield.calculate_bond_force(molecule.bond)
            molecule.atom1.add_force(force)
            molecule.atom2.add_force(-force)

        # Calculate non-bonded forces between molecules
        for i, mol1 in enumerate(self.molecules):
            for mol2 in self.molecules[i+1:]:
                # Calculate forces between atoms of different molecules
                for atom1 in [mol1.atom1, mol1.atom2]:
                    for atom2 in [mol2.atom1, mol2.atom2]:
                        force = self.forcefield.calculate_lj_force(atom1, atom2)
                        atom1.add_force(force)
                        atom2.add_force(-force)

    # def update_positions_and_velocities(self):
```

The constructor of the `MDSimulation` class now takes a list of molecules as an argument. The list of atoms is created by flattening the list of atoms in each molecule. This is all we need in the constructor.

The `calculate_forces` method now calculates the bond forces for each molecule and adds them to the atoms. The non-bonded forces are calculated as before. The method`update_positions_and_velocities` will remain the same as we only need to add the bond forces to the total force calculation.


## Create a diatomic molecule

Finally, we need something to create the diatomic molecules for out simulation. We will write a function that creates a number of diatomic molecules with a given box size and atom types. The function will create the atoms at random positions with a small displacement from the center of a grid and create a bond between them.


```python
```python
def create_diatomic_molecules(num_molecules, box_size, type1="H", type2="H", mass1=1.0, mass2=1.0):
    molecules = []
    spacing = np.min(box_size) / np.ceil(np.sqrt(num_molecules))

    for i in range(num_molecules):
        # Calculate grid position for molecule center
        row = i // int(np.ceil(np.sqrt(num_molecules)))
        col = i % int(np.ceil(np.sqrt(num_molecules)))
        center = np.array([col * spacing + spacing/2, row * spacing + spacing/2])

        # Create atoms with small random displacement for initial bond length
        displacement = np.random.randn(2) * 0.1
        atom1 = Atom(2*i, type1, center + displacement, mass=mass1)
        atom2 = Atom(2*i+1, type2, center - displacement, mass=mass2)

        # Create bond
        ff = ForceField()
        bond_params = ff.bond_parameters[(type1, type2)]
        bond = Bond(atom1, atom2, bond_params['k'], bond_params['r0'])

        molecules.append(DiatomicMolecule(atom1, atom2, bond))

    return molecules
```


## MD Simulation


In the last step we need to initialize the simulation with the diatomic molecules and run the simulation.

```python
## Initialize simulation as before

box_size = np.array([50.0, 50.0])
num_molecules = 100
molecules = create_diatomic_molecules(num_molecules, box_size, "H", "H")
ff = ForceField()
sim = MDSimulation(molecules, ff, 0.001, box_size)

# Initialize velocities for all atoms
atoms = [atom for mol in molecules for atom in [mol.atom1, mol.atom2]]
initialize_velocities(atoms, temperature=5.0)

## rest of the simulation goes here
```

## Summary

In this module, we extended our molecular dynamics simulation to include diatomic molecules with bond vibrations. We added several new components:

1. A Bond class to model the harmonic bond between two atoms, implementing Hooke's law for the bond force
2. A DiatomicMolecule class to manage pairs of bonded atoms
3. Updates to the ForceField class to include bond parameters and force calculations
4. Modifications to the MDSimulation class to handle both bonded and non-bonded interactions

The bond vibrations are modeled as harmonic oscillators with a spring constant k and equilibrium bond length $r_0$. The total forces on each atom now include both the van der Waals forces between molecules and the bond forces within molecules. We provided functions to create and initialize diatomic molecules in a simulation box, setting up a foundation for simulating more complex molecular systems.

The implementation maintains the modular structure of our previous code while adding the capability to simulate molecular vibrations, bringing us closer to modeling real molecular systems.